In [1]:
#Importing libraries
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [2]:
#importing stock list
stocks =pd.read_csv('sp_500_stocks.csv')

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

In [4]:
symbol = 'AAPL'
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'ARTO)/G(LNE EDSSEMNBLLA QKGA TACS',
 'calculationPrice': 'close',
 'open': 133.75,
 'openTime': 1639825011334,
 'openSource': 'lcifoafi',
 'close': 130.25,
 'closeTime': 1673906726663,
 'closeSource': 'oilifcfa',
 'high': 132.69,
 'highTime': 1683899841076,
 'highSource': '15ide uenlta ypircedem ',
 'low': 130.77,
 'lowTime': 1673177313960,
 'lowSource': 'ied r cldi1ee ntypeuma5',
 'latestPrice': 127.58,
 'latestSource': 'Close',
 'latestTime': 'May 10, 2021',
 'latestUpdate': 1665649879347,
 'latestVolume': 89515703,
 'iexRealtimePrice': 128.43,
 'iexRealtimeSize': 43,
 'iexLastUpdated': 1687442486133,
 'delayedPrice': 131.965,
 'delayedPriceTime': 1668060313781,
 'oddLotDelayedPrice': 128.66,
 'oddLotDelayedPriceTime': 1627580171639,
 'extendedPrice': 131.41,
 'extendedChange': -0.56,
 'extendedChangePercent': -0.00431,
 'extendedPriceTime': 1625937577173,
 'previousClose': 130.5,
 'previousVolume': 81508365,
 'chan

In [5]:
price =data['latestPrice']
market_cap = data['marketCap']
market_cap/1000000000000

2.19792878728

In [6]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [7]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index=my_columns
    ),
   
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,127.58,2197928787280,N/A


In [8]:
final_dataframe =pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker']:
    api_url = f"https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
    pd.Series(
    [
        stock,
        data['latestPrice'],
        data['marketCap'],
        'N/A'
    ],
    index=my_columns
    ),
   
    ignore_index=True,
)

In [9]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,138.33,40542073642,N/A
1,AAL,22.00,14773970178,N/A
2,AAP,214.32,14308443032,N/A
3,AAPL,131.64,2182538773847,N/A
4,ABBV,116.51,209831935741,N/A
...,...,...,...,...
500,YUM,123.90,37486676975,N/A
501,ZBH,179.68,37271911418,N/A
502,ZBRA,496.19,26595677754,N/A
503,ZION,61.10,9653810374,N/A


In [10]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [11]:
symbol_groups= list(chunks(stocks['Ticker'],100))
symbol_strings=[]
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url=f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
#     print(batch_api_call_url)
    data= requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe=final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['marketCap'],
            'N/A'
        ],
        index=my_columns),
        ignore_index=True
        )
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,136.82,40821450671,N/A
1,AAL,22.00,14401367666,N/A
2,AAP,215.73,13752071167,N/A
3,AAPL,127.57,2197570701348,N/A
4,ABBV,121.85,209150917316,N/A
...,...,...,...,...
500,YUM,127.14,36708093377,N/A
501,ZBH,180.54,37555870579,N/A
502,ZBRA,488.80,26453123353,N/A
503,ZION,59.72,9730704298,N/A


In [23]:
portfolio_size = input('Enter the value of your portfolio:')
try:
    val = float(portfolio_size)
    print(val)
except ValueError:
    print("That's not a number! \nPlease try again")
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)

Enter the value of your portfolio:10000000
10000000.0


In [29]:
position_size=val/len(final_dataframe.index)
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])
final_dataframe
# number_of_apple_shares = position_size/500
# math.floor(number_of_apple_shares)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,136.82,40821450671,144
1,AAL,22.00,14401367666,900
2,AAP,215.73,13752071167,91
3,AAPL,127.57,2197570701348,155
4,ABBV,121.85,209150917316,162
...,...,...,...,...
500,YUM,127.14,36708093377,155
501,ZBH,180.54,37555870579,109
502,ZBRA,488.80,26453123353,40
503,ZION,59.72,9730704298,331


In [80]:
writer=pd.ExcelWriter('recommended trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index=False)

In [81]:
background_color='#0a0a23'
font_color='#ffffff'

string_format = writer.book.add_format(
    {
    'font_color': font_color,
    'bg_color':background_color,
    'border':1
    }
)

dollar_format = writer.book.add_format(
    {
    'num_format': '$0.00',
    'font_color': font_color,
    'bg_color':background_color,
    'border':1
    }
)

integer_format = writer.book.add_format(
    {
    'num_format':'0',
    'font_color': font_color,
    'bg_color':background_color,
    'border':1
    }
)

In [82]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
# writer.save()
# writer.sheets['Recommended Trades'].write('A1','Ticker',string_format)
# writer.sheets['Recommended Trades'].write('B1','Stock Price',dollar_format)
# writer.sheets['Recommended Trades'].write('C1','Market Capitalization',dollar_format)
# writer.sheets['Recommended Trades'].write('D1','No. of Shares to Buy',integer_format)

In [83]:
column_formats={
    'A':['Ticker', string_format],
    'B':['Stock Price', dollar_format],
    'C':['Market Capitalization', dollar_format],
    'D':['No. of Shares to Buy', integer_format]
}
for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1',column_formats[column][0], column_formats[column][1])
writer.save()